In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd

# Load file CSV dari Google Drive
df = pd.read_csv('/content/drive/MyDrive/PBA/Link Berita Kelompok 9.csv')

# Cek isi awal
df.head()

,No,PIC,Link
0,1,Nala,https://money.kompas.com/read/2025/08/12/15391...
1,2,NaN,https://regional.kompas.com/read/2025/07/08/10...
2,3,NaN,https://video.kompas.com/watch/1858879/ombak-h...
3,4,NaN,https://money.kompas.com/read/2025/05/23/22053...
4,5,NaN,https://travel.kompas.com/read/2025/05/16/0707...


In [ ]:
!pip install newspaper3k

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 53.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.1/211.1 kB 16.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.5/81.5 kB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.4/107.4 kB 9.2 MB/s eta 0:00:00
  Created wheel for tinysegmenter: filename=tinysegmenter-0.3-py3-none-any.whl size=13540 sha256=66b5d36df48bdbd5a8b3c007c253c7e57b4c43d0accfef429a1e83dd5f70472a
  Stored in directory: /root/.cache/pip/wheels/a5/91/9f/00d66475960891a64867914273fcaf78df6cb04d905b104a2a
  Created wheel for feedfinder2: filename=feedfinder2-0.0.4-py3-none-any.whl size=3341 sha256=4f477729516cb08105117114b58615b2ef0ac5a2ef7e89ca82a33be94d08c0bb
  Stored in directory: /root/.cache/pip/wheels/9f/9f/fb/364871d7426d3cdd4d293dcf7e53d97f16

In [ ]:
!pip install lxml_html_clean

In [ ]:
from newspaper import Article
import pandas as pd
import time

def scrape_berita(url):
    try:
        artikel = Article(url, language='id')
        artikel.download()
        artikel.parse()
        return pd.Series({
            'judul': artikel.title,
            'tanggal': artikel.publish_date,
            'konten': artikel.text,
            'label': None  # Placeholder untuk anotasi sentimen nanti
        })
    except Exception as e:
        print(f"Gagal scrape: {url}\nError: {e}")
        return pd.Series({
            'judul': None,
            'tanggal': None,
            'konten': None,
            'label': None
        })

# Terapkan ke semua link
hasil_scrape = df['Link'].apply(scrape_berita)

# Gabungkan ke dataframe awal
df_final = pd.concat([df, hasil_scrape], axis=1)

# Lihat hasil awal
df_final.head()

,No,PIC,Link,judul,tanggal,konten,label
0,1,Nala,https://money.kompas.com/read/2025/08/12/15391...,ASDP dan Operator Ferry Swasta Perkuat Data Ma...,2025-08-12 00:00:00,"JAKARTA, KOMPAS.com — Manifest penumpang dan k...",None
1,2,NaN,https://regional.kompas.com/read/2025/07/08/10...,"Listrik di Pulau Enggano Sudah 24 Jam, Kapal F...",2025-07-08 00:00:00,"BENGKULU, KOMPAS.com - Kepala Desa Kaana, Alam...",None
2,3,NaN,https://video.kompas.com/watch/1858879/ombak-h...,"Ombak Hingga 2,5 Meter Hambat Pencarian KMP Tu...",2025-07-03 02:09:12+00:00,"KMP Tunu Pratama Jaya tenggelam di Selat Bali,...",None
3,4,NaN,https://money.kompas.com/read/2025/05/23/22053...,Rute Ferry Batam–Johor Direncanakan Aktif Tahu...,2025-05-23 00:00:00,KOMPAS.com – PT Angkutan Sungai Danau dan Peny...,None
4,5,NaN,https://travel.kompas.com/read/2025/05/16/0707...,"Menyeberangi Danau Toba, Perjalanan Naik Kapal...",2025-05-16 00:00:00,KOMPAS.com - Kala melihat kapal berwarna hijau...,None


# PREPROCESSING

In [ ]:
import re
import string
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
import re
import string
import nltk
from nltk.corpus import stopwords
# Download necessary NLTK resources
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('punkt_tab')

def bersihkan_teks(teks):
    if pd.isnull(teks):
        return ""
    # Lowercase
    teks = teks.lower()
    # Hilangkan angka dan tanda baca
    teks = re.sub(r'\d+', '', teks)
    teks = teks.translate(str.maketrans('', '', string.punctuation))
    # Tokenisasi
    tokens = nltk.word_tokenize(teks)
    # Stopword removal
    stop_words = set(stopwords.words('indonesian'))
    tokens = [kata for kata in tokens if kata not in stop_words]
    # Gabungkan kembali
    return ' '.join(tokens)

# Terapkan ke kolom konten
df_final['konten_bersih'] = df_final['konten'].apply(bersihkan_teks)

# Lihat hasil
df_final[['judul', 'konten_bersih']].head()

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


,judul,konten_bersih
0,ASDP dan Operator Ferry Swasta Perkuat Data Ma...,jakarta kompascom — manifest penumpang kendara...
1,"Listrik di Pulau Enggano Sudah 24 Jam, Kapal F...",bengkulu kompascom kepala desa kaana alamudin ...
2,"Ombak Hingga 2,5 Meter Hambat Pencarian KMP Tu...",kmp tunu pratama jaya tenggelam selat bali rab...
3,Rute Ferry Batam–Johor Direncanakan Aktif Tahu...,kompascom – pt angkutan sungai danau penyebera...
4,"Menyeberangi Danau Toba, Perjalanan Naik Kapal...",kompascom kapal berwarna hijau bersandar pelab...


In [ ]:
from collections import Counter

# Gabungkan semua teks
gabungan = ' '.join(df_final['konten_bersih'].dropna())
kata_kata = gabungan.split()
frekuensi = Counter(kata_kata)

# Tampilkan 20 kata paling sering
frekuensi.most_common(20)

[('kapal', 1795),
 ('rp', 1691),
 ('kendaraan', 1342),
 ('penumpang', 1150),
 ('pelabuhan', 1066),
 ('tiket', 1055),
 ('feri', 879),
 ('golongan', 879),
 ('penyeberangan', 655),
 ('wita', 633),
 ('jadwal', 616),
 ('asdp', 613),
 ('kmp', 581),
 ('rute', 556),
 ('keberangkatan', 476),
 ('barang', 473),
 ('ferry', 444),
 ('a', 435),
 ('b', 427),
 ('harga', 406)]

In [ ]:
output_filename = 'df_final.csv'
df_final.to_csv(output_filename, index=False, sep=';')

from google.colab import files
files.download(output_filename)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>